In [16]:
import os
import sys

sys.path.append('../lightningmodule')
from sklearn.model_selection import GroupKFold
from utils.utils import get_sorted_files_by_type, set_seed
from constants import CLASSES, CLASS2IND
import torch
import numpy as np
import json
import cv2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [17]:
def load_data(pngs, jsons, K=1, valid_idx=1):

   assert valid_idx <= K

   _filenames = np.array(pngs)
   _labelnames = np.array(jsons)

   train_datalist = dict(filenames = [], labelnames = [])
   valid_datalist = dict(filenames = [], labelnames = [])

   train_datalist['filenames'] += list(_filenames)
   train_datalist['labelnames'] += list(_labelnames)

   return train_datalist, valid_datalist

In [18]:
handbone = ['Hamate', 'Scaphoid', 'Lunate', 'Trapezium', 'Capitate', 'Triquetrum', 'Trapezoid', 'Pisiform']

In [19]:
def crop_img_custom(image, annotations, handbone):
    # handbone에 해당하는 annotation만 필터링
    filtered_annotations = [ann for ann in annotations if ann["label"] in handbone]
    
    # 필터링된 annotation의 모든 포인트를 모음
    all_points = []
    for ann in filtered_annotations:
        points = ann["points"]
        all_points.extend(points)
    
    all_points = np.array(all_points)
    x_coords = all_points[:, 0]
    y_coords = all_points[:, 1]

    # 최소 및 최대 x, y 좌표 계산
    min_x, min_y = int(np.min(x_coords)), int(np.min(y_coords))
    max_x, max_y = int(np.max(x_coords)), int(np.max(y_coords))
    
    h, w = image.size[1], image.size[0]  
    crop_w, crop_h = max_x - min_x, max_y - min_y
    
    # 이미지 크기에 따라 여백 비율 조정
    width_margin = int((w - crop_w) * 0.005)  
    height_margin = int((h - crop_h) * 0.005)  

    # 여백을 포함하여 새로운 크롭 좌표 설정
    new_min_x = max(0, min_x - width_margin)
    new_max_x = min(w, max_x + width_margin)
    new_min_y = max(0, min_y - height_margin)
    new_max_y = min(h, max_y + height_margin)
    print(new_min_x, new_max_x, new_min_y, new_max_y)

    # 이미지 크롭
    image = image.crop([new_min_x, new_min_y, new_max_x, new_max_y])

    # 필터링된 annotation points 좌표 수정
    for ann in filtered_annotations:
        new_points = []
        for point in ann["points"]:
            new_x = point[0] - new_min_x
            new_y = point[1] - new_min_y
            new_points.append([new_x, new_y])
        ann["points"] = new_points

    return image, filtered_annotations



In [21]:
TRAIN_DATA_DIR = '/data/ephemeral/home/data/train_crop'
image_root = os.path.join(TRAIN_DATA_DIR, 'DCM')
label_root = os.path.join(TRAIN_DATA_DIR, 'outputs_json')
pngs = get_sorted_files_by_type(image_root, 'png')
jsons = get_sorted_files_by_type(label_root, 'json')
train_files, valid_files = load_data(pngs, jsons)
save_path = './LR_cropped'

for img_path, label_path in zip(train_files['filenames'], train_files['labelnames']):
    # 이미지 로드
    image = Image.open(img_path)
    
    # 라벨 로드
    with open(label_path, "r") as f:
        data = json.load(f)
    
    # handbone 클래스만 선택하여 이미지 크롭 및 좌표 수정
    cropped_image, filtered_annotations = crop_img_custom(image, data["annotations"], handbone)
    
    # 이미지 패딩
    padded_image, x_offset, y_offset = pad_image(cropped_image)
    
    # annotation points 좌표 수정 (패딩 offset 적용)
    for ann in filtered_annotations:
        new_points = []
        for point in ann["points"]:
            new_x = point[0] + x_offset
            new_y = point[1] + y_offset
            new_points.append([new_x, new_y])
        ann["points"] = new_points
        
    # annotation points 좌표 4배 확대
    scale_factor = 4
    max_size = max(padded_image.size)
    new_size = max_size * scale_factor
    
    for ann in filtered_annotations:
        new_points = []
        for point in ann["points"]:
            new_x = point[0] * scale_factor
            new_y = point[1] * scale_factor
            new_points.append([new_x, new_y])
        ann["points"] = new_points
    
    # metadata 크기 수정 및 필터링된 annotation만 저장
    data["annotations"] = filtered_annotations
    data["metadata"]["width"] = new_size
    data["metadata"]["height"] = new_size
    
    # 이미지 저장
    padded_image.save(img_path)
    
    # 라벨 저장
    with open(label_path, "w") as f:
        json.dump(data, f, indent=2)
        
    print(f"Processed: {img_path}")
    print(f"Original size: {image.size}")
    print(f"Cropped size: {cropped_image.size}")
    print(f"Padded size: {padded_image.size}")
    print(f"Final annotation size: {new_size}x{new_size}")
    print(f"Number of annotations: {len(filtered_annotations)}")

786 1132 1210 1474
Processed: ../../../data/train_crop/DCM/ID001/image1661130828152_R.png
Original size: (2048, 2048)
Cropped size: (346, 264)
Padded size: (346, 346)
Final annotation size: 1384x1384
Number of annotations: 8
793 1159 1289 1551
Processed: ../../../data/train_crop/DCM/ID001/image1661130891365_L.png
Original size: (2048, 2048)
Cropped size: (366, 262)
Padded size: (366, 366)
Final annotation size: 1464x1464
Number of annotations: 8
815 1216 1377 1645
Processed: ../../../data/train_crop/DCM/ID002/image1661144206667.png
Original size: (2048, 2048)
Cropped size: (401, 268)
Padded size: (401, 401)
Final annotation size: 1604x1604
Number of annotations: 8
829 1217 1386 1652
Processed: ../../../data/train_crop/DCM/ID002/image1661144246917.png
Original size: (2048, 2048)
Cropped size: (388, 266)
Padded size: (388, 388)
Final annotation size: 1552x1552
Number of annotations: 8
814 1171 1274 1500
Processed: ../../../data/train_crop/DCM/ID003/image1661144310162.png
Original size: (

In [ ]:
TRAIN_DATA_DIR = '/data/ephemeral/home/data/train_crop'
image_root = os.path.join(TRAIN_DATA_DIR, 'DCM')
label_root = os.path.join(TRAIN_DATA_DIR, 'outputs_json')
pngs = get_sorted_files_by_type(image_root, 'png')
img_name = []
sr_name = []
for i in pngs:
    img_name.append(i.split('/')[-1].split('.png')[0])




sr_data = 'results/HMA_test_X4/visualization/Test'
image_root = os.path.join(sr_data)
sr_pngs = get_sorted_files_by_type(image_root, 'png')

for i in sr_pngs:
    sr_name.append(i.split('/')[-1].split('_HMA')[0])

def find_indices(list1, list2):
    # list2의 각 값과 인덱스를 딕셔너리로 만듦
    index_dict = {value: index for index, value in enumerate(list2)}
    # list1의 각 값에 대해 list2에서의 인덱스를 찾음
    return [index_dict[value] for value in list1]

match_name = find_indices(img_name,sr_name)

for idx in range(len(pngs)):
    if img_name[i]!=sr_name[match_name[i]]:
        break
    else:
        !mv {sr_pngs[match_name[i]]} {pngs[i]}
print(match_name)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [38]:

list1=[1,2,3,4]
list2=[3,4,2,1]
match_name = find_indices(list1, list2)
print(match_name)

[3, 2, 0, 1]


In [ ]:
3,2,0,1